In [ ]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 449.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.8/871.8 kB 22.0 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import os
import torch

In [ ]:
device  = "cuda" if torch.cuda.is_available() else "cpu"
# device  = "cpu"
device

'cuda'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Загрузка модели YOLOv8

madel_name = 'yolov8n.pt'
# madel_name = 'yolov8s.pt'
# madel_name = 'yolov8m.pt'
# madel_name = 'yolov8l.pt'
# madel_name = 'yolov8x.pt'



model = YOLO(madel_name).to(device)

100%|██████████| 6.25M/6.25M [00:00<00:00, 103MB/s]


# Работа с отдельными изображениями

In [ ]:
paths = ['/content/drive/MyDrive/Colab Notebooks/Практика/cat_dog.jpg',
         '/content/drive/MyDrive/Colab Notebooks/Практика/img.jpg',
         '/content/drive/MyDrive/Colab Notebooks/Практика/picture.jpg']

In [ ]:
results = model.predict(paths)
result = results[2]


0: 640x640 1 cat, 1 dog, 1 sports ball, 16.3ms
1: 640x640 4 persons, 3 cars, 1 truck, 3 traffic lights, 2 stop signs, 1 sports ball, 16.3ms
2: 640x640 2 persons, 1 refrigerator, 16.3ms
Speed: 10.7ms preprocess, 16.3ms inference, 283.0ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
result.boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([ 0.,  0., 72.], device='cuda:0')
conf: tensor([0.8053, 0.6001, 0.2866], device='cuda:0')
data: tensor([[5.6038e+01, 6.4117e+01, 1.4136e+02, 2.8881e+02, 8.0534e-01, 0.0000e+00],
        [8.2902e+02, 4.2061e+01, 8.6678e+02, 8.8082e+01, 6.0011e-01, 0.0000e+00],
        [6.7753e+02, 6.3238e+01, 8.4116e+02, 2.8857e+02, 2.8657e-01, 7.2000e+01]], device='cuda:0')
id: None
is_track: False
orig_shape: (523, 965)
shape: torch.Size([3, 6])
xywh: tensor([[ 98.6975, 176.4619,  85.3195, 224.6897],
        [847.8989,  65.0717,  37.7643,  46.0204],
        [759.3417, 175.9038, 163.6312, 225.3317]], device='cuda:0')
xywhn: tensor([[0.1023, 0.3374, 0.0884, 0.4296],
        [0.8787, 0.1244, 0.0391, 0.0880],
        [0.7869, 0.3363, 0.1696, 0.4308]], device='cuda:0')
xyxy: tensor([[ 56.0378,  64.1171, 141.3572, 288.8067],
        [829.0167,  42.0615, 866.7811,  88.0819],
        [677.5261,  63.2380, 841.1573, 288.5697]], device='cuda:0

In [ ]:
result.boxes.cls

tensor([ 0.,  0., 72.], device='cuda:0')

In [ ]:
classes = result.boxes.cls.tolist()
classes

[16.0, 15.0, 32.0]

In [ ]:
classes.count(16.0)

1

### Вывод изображения

In [ ]:
# изменение и сохранение нового изображения
from PIL import Image
for i, r in enumerate(results):
	img_bgr = r.plot()
	im_rgb = Image.fromarray(img_bgr[...,::-1])
	# r.show()
	r.save(filename = f"results{i}.jpg")

# Работа с видео

In [ ]:
# Открытие исходного видеофайла
# mini-Trim.mkv --- factory_cam.mkv
input_video_path = '/content/drive/MyDrive/Colab Notebooks/Практика/mini2.mkv'
capture = cv2.VideoCapture(input_video_path)

In [ ]:
# Чтение параметров видео
fps = int(capture.get(cv2.CAP_PROP_FPS))
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Настройка выходного файла
output_video_path = f'/content/drive/MyDrive/Colab Notebooks/Практика/detect4_by_{madel_name}.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

In [ ]:
fps

48

#### Обработка видеофайла

In [ ]:
sec_arr = []
cnt = 0
persons = []
import time
import statistics
start_time = time.time()

color = (255, 0, 0) #BGR

while True:
    # Захват кадра
    ret, frame = capture.read()
    if not ret:
        break

    # Обработка кадра с помощью модели YOLO
    results = model(frame)[0]

    # Получение данных об объектах
    classes_names = results.names
    classes = results.boxes.cls.cpu().numpy()
    boxes_xy = results.boxes.xyxy.cpu().numpy().astype(np.int32)

    # количество человек в кадре
    p_on_frame = classes.tolist().count(0.0)
    persons.append(p_on_frame)

    # Рисование рамок и подписей на кадре
    for class_id, box, conf in zip(classes, boxes_xy, results.boxes.conf):
        if class_id == 0.0:
            # название класса
            class_name = classes_names[int(class_id)]
            # рамки
            x1, y1, x2, y2 = box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            cv2.putText(frame, f'{class_name} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    # количество людей на кадре
    cv2.putText(frame, f'Persons: {p_on_frame}', (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    if cnt == fps:
       # подсчёт среднего количества человек на кадрах в данной секунде видео и сохранение результата
        sec_arr.append(round(statistics.mean(persons)))
        cnt = 0
        persons = []

    # Запись обработанного кадра в выходной файл
    writer.write(frame)
    cnt +=1

# Освобождение ресурсов и закрытие окон
capture.release()
writer.release()

end_time = time.time()


0: 384x640 1 truck, 94.2ms
Speed: 17.1ms preprocess, 94.2ms inference, 755.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 8.5ms
Speed: 4.0ms preprocess, 8.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 7.0ms
Speed: 3.6ms preprocess, 7.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 8.7ms
Speed: 3.3ms preprocess, 8.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 8.3ms
Speed: 3.4ms preprocess, 8.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 8.9ms
Speed: 2.8ms preprocess, 8.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 8.8ms
Speed: 2.8ms preprocess, 8.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 7.5ms
Speed: 3.4ms preprocess, 7.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640

In [ ]:
print(f"Время обработки видео mini2.mkv моделью {madel_name}: {(end_time - start_time):.2f}")

Время обработки видео mini2.mkv моделью yolov8n.pt: 59.57


Время обработки видео mini2.mkv моделью yolov8n.pt: 55.62 </br>
Время обработки видео mini2.mkv моделью yolov8s.pt: 57.89 </br>
Время обработки видео mini2.mkv моделью yolov8m.pt: 66.79 </br>
Время обработки видео mini2.mkv моделью yolov8l.pt: 66.82 </br>
Время обработки видео mini2.mkv моделью yolov8x.pt: 79.37 </br>
</br>
</br>
Время обработки видео factory_cam.mkv моделью yolov8x.pt: 207.49 </br>
Время обработки видео factory_cam.mkv моделью yolov8s.pt: 146.92 </br>
Время обработки видео factory_cam.mkv моделью yolov8n.pt: 140.81


In [ ]:
for i in range(len(sec_arr)):
  print(f'Секунда: {i+1}, количество человек: {sec_arr[i]}')

Секунда: 1, количество человек: 0
Секунда: 2, количество человек: 0
Секунда: 3, количество человек: 0
Секунда: 4, количество человек: 1
Секунда: 5, количество человек: 1
Секунда: 6, количество человек: 1
Секунда: 7, количество человек: 1
Секунда: 8, количество человек: 1
Секунда: 9, количество человек: 1
Секунда: 10, количество человек: 1
Секунда: 11, количество человек: 1
Секунда: 12, количество человек: 1
Секунда: 13, количество человек: 3
Секунда: 14, количество человек: 3
Секунда: 15, количество человек: 3
Секунда: 16, количество человек: 2
Секунда: 17, количество человек: 2
Секунда: 18, количество человек: 2
Секунда: 19, количество человек: 3
Секунда: 20, количество человек: 3
Секунда: 21, количество человек: 4
Секунда: 22, количество человек: 2
Секунда: 23, количество человек: 1


### Сравнение времени работы моделей

In [ ]:
models = ['yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt', 'yolov8x.pt']
times = []


for model_name in models:
    model1 = YOLO(model_name).to(device)
    input_video_path = '/content/drive/MyDrive/Colab Notebooks/Практика/factory_cam.mkv'
    capture = cv2.VideoCapture(input_video_path)


    sec_arr = []
    cnt = 0
    persons = []
    import time
    import statistics
    start_time = time.time()

    color = (255, 0, 0) #BGR

    while True:
        # Захват кадра
        ret, frame = capture.read()
        if not ret:
            break

        # Обработка кадра с помощью модели YOLO
        results = model1(frame)[0]

        # Получение данных об объектах
        classes_names = results.names
        classes = results.boxes.cls.cpu().numpy()
        boxes_xy = results.boxes.xyxy.cpu().numpy().astype(np.int32)

        # количество человек в кадре
        p_on_frame = classes.tolist().count(0.0)
        persons.append(p_on_frame)

        if cnt == fps:
          # подсчёт среднего количества человек на кадрах в данной секунде видео и сохранение результата
            sec_arr.append(round(statistics.mean(persons)))
            cnt = 0
            persons = []

        cnt +=1

    # Освобождение ресурсов и закрытие окон
    capture.release()


    end_time = time.time()
    times.append(f"Время обработки видео factory_cam.mkv моделью {model_name}: {(end_time - start_time):.2f}")

Выходные данные были обрезаны до нескольких последних строк (5000).
0: 384x640 2 persons, 4 tvs, 33.8ms
Speed: 3.0ms preprocess, 33.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 tvs, 33.2ms
Speed: 2.4ms preprocess, 33.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 tvs, 33.2ms
Speed: 2.7ms preprocess, 33.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 tvs, 33.6ms
Speed: 2.8ms preprocess, 33.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 tvs, 32.9ms
Speed: 3.4ms preprocess, 32.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 tvs, 34.0ms
Speed: 3.2ms preprocess, 34.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 tvs, 34.1ms
Speed: 2.6ms preprocess, 34.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 

In [ ]:
times

['Время обработки видео factory_cam.mkv моделью yolov8n.pt: 103.90',
 'Время обработки видео factory_cam.mkv моделью yolov8s.pt: 109.77',
 'Время обработки видео factory_cam.mkv моделью yolov8m.pt: 118.19',
 'Время обработки видео factory_cam.mkv моделью yolov8l.pt: 133.43',
 'Время обработки видео factory_cam.mkv моделью yolov8x.pt: 165.24']

Для mini2.mkv: </br>
['Время обработки моделью yolov8n.pt: 39.33', </br>
 'Время обработки моделью yolov8s.pt: 44.69',</br>
 'Время обработки моделью yolov8m.pt: 49.51',</br>
 'Время обработки моделью yolov8l.pt: 52.54',</br>
 'Время обработки моделью yolov8x.pt: 64.76']</br>